In [1]:
# добавить в класс db_link
def get_table_cond(self, t, col, value):
    #   print ( conn.get_dsn_parameters(),"\n")
        if type(value)=='str':
             query = "SELECT * from e_and_s_schema.%s where %s=''%s''" %(t, col, value)
        else:
            query = "SELECT * from e_and_s_schema.%s where %s=%s" %(t, col, value)
        df_results = pd.read_sql_query(query, self.conn)
        return df_results

    
class user_anketa():
    
    def __init__(self, route_id, conn_obj):
        
        self.route_id = route_id
        
        self.anketa=self.get_from_route_tbl(conn_obj)
        
        self.user_id  = self.anketa.iloc[0]['user_id']
        
        self.visit_time =  int ( self.anketa.iloc[0]['prof_type'] )
        self.wake_up_time = int (self.anketa.iloc[0]['wake_up_time']) +6 
        self.n_days = int( self.anketa.iloc[0]['count_days'] )
        
        
        if self.anketa.iloc[0]['date_start']  is None:
            self.first_day = now
        else:
            self.first_day = self.anketa.iloc[0]['date_start']
        
#         self.last_day = self.first_day + datetime.timedelta (days=self.n_days)
        
        self.list_interests = self.get_interests(conn_obj)
        self.start_point = self.get_location(conn_obj)
        self.list_selected_places = self.get_selected_places(conn_obj)  

    def get_from_route_tbl (self, conn_obj):
#         try:
        _ = conn_obj.open_conn()
        df = conn_obj.get_table_cond('route_tbl', 'route_id',self.route_id)
        _ = conn_obj.close_conn()
         
        return df
       
#         except:
            
#             return 0
    def get_location(self, conn_obj):
        
        _ = conn_obj.open_conn()
        df = conn_obj.get_table_cond('user_tbl', 'user_id', self.user_id)
        _ = conn_obj.close_conn()
        lat = df.iloc[0]['gps_lat']
        long = df.iloc[0]['gps_long']
        
        if lat==0 or long==0 or lat is None or long is None :
            return red_square_coords
        else:
            return lat, long

    def get_interests(self, conn_obj):
        
        _ = conn_obj.open_conn()
        df = conn_obj.get_table_cond('user_categories_of_interests_tbl', 'user_id', self.user_id)
        _ = conn_obj.close_conn()
        list_v=  df['category_of_interest_id'].values
        
        return list_v
    
    def get_selected_places(self, conn_obj):
    
        _ = conn_obj.open_conn()
        df = conn_obj.get_table_cond('route_object_tbl', 'route_id', self.route_id)
        _ = conn_obj.close_conn()
        list_v = df.loc[df['recommend_type']=='user choice']['showplace_id'].values
        return list_v

class itinerary():
    
    def __init__(self, ua):
        
        self.itinerary = dict()
        self.obj_num = ua.n_days*3
        self.final_list = []
        self.object_dict=dict()
        
    def get_start(self, ua):
        
        if ua.first_day.hour<16:
            
            self.start_day=ua.first_day.date()
            self.start_hour=ua.first_day.hour+1
        else:
            
            self.start_day=ua.first_day.date()+datetime.timedelta(days=1)
            self.start_hour=ua.wake_up_time
        
#         return start_day, start_hour
    def get_adj_mat(self,):
        adj_mat=nx.to_pandas_adjacency(self.G)
        dic_col={i:adj_mat.loc[adj_mat[i]!=0][i].mean()*math.pow(10,self.object_dict[i]) for i in adj_mat.columns}
        sorted_cols=list({k: v for k, v in sorted(dic_col.items(), key=lambda item: 100000 if item[1]/item[1]!=1  else item[1])}.keys())
        
        return adj_mat
    
    def get_dict_days(self, ua, dens_obj):
        
        adj_mat=self.get_adj_mat()
        
        for n_day in range( ua.n_days):
#             print(n_day)
            day = self.start_day + datetime.timedelta(days=n_day)
            self.itinerary [day] = list()
            left_list = adj_mat.columns
        #     print(left_list)
            if n_day==0:
                start_hour=self.start_hour
            else:
                start_hour= ua.wake_up_time

            start_cur_hour=start_hour
            start_cur_min=0
            order=1
#             print('start_vis_hour',start_vis_hour)
            while start_cur_hour< 17 and len(adj_mat.columns)>0 :
#                 print(order)
                if order!=1:
        #             print(adj_mat)
                    show_id, time_to_get = self.get_next_place(show_id, adj_mat, day, start_vis_hour, dens_obj)
                    adj_mat= adj_mat.loc[left_list][left_list]

                else:

                    time_to_get=60 # должна быть функиця расстояния от тек местоположения
                    adj_mat= adj_mat.loc[left_list][left_list]
                    show_id=adj_mat.columns[0]  

        #         print('куда как долго',show_id, time_to_get)
                start_vis_hour, start_vis_min, start_time_to_show= self.get_hour_min_start(start_cur_hour, start_cur_min,time_to_get)
               
                
                vis_time = int( min( ua.visit_time*60*2/3, ( (18 - start_vis_hour)*60 - start_vis_min )*2/3 ) )


                if start_vis_hour>18:
                    is_active=0 
                else:
                    is_active=1
                
                self.itinerary [day].append(
                    {
                    'id':show_id,
                    'order':order,
                    'time to get': time_to_get ,
                    'time_to_start':  start_time_to_show,
                    'visit_time':vis_time ,
                    'comments': '',
                    'is_active': is_active,
                    'day':n_day+1
                    }
                )
#                 print(self.itinerary [day])
                order+=1
    
                start_cur_hour, start_cur_min, _= self.get_hour_min_start(start_vis_hour, start_vis_min, vis_time)

                left_list=[ i for i in left_list if i != show_id]
        #         print(left_list)
        #     self.obj_num+=order
            
    def get_random_dist(self):
        
        self.list_dist = list_distance
        h_v = len(self.list_dist)-1
        return self.list_dist[rnd.randint(0,h_v)]   
    
    def get_dist_between_start(start_location):
        return 0
            
    def get_hour_min_start(self, start_hour, start_min, time_to_get):
        
        start_vis_min = int((start_min+time_to_get%60)%60)
        ost = int((start_min+time_to_get%60)//60)
        start_vis_hour = int(start_hour+time_to_get//60+ost)
    
        if len(str(start_vis_min))<2:
            
            str_min='0'+str(start_vis_min)
        else:
            str_min=str(start_vis_min)

        start_time_to_show=str(start_vis_hour)+':'+str_min
        
        return start_vis_hour,start_vis_min, start_time_to_show
    
    def get_list_ob(self, selected_list, list_interests):
           
        self.object_dict = dict(zip(selected_list,[1]*len(selected_list)))
        
#         print(self.object_dict,len(self.object_dict))
        
        if len(self.object_dict)>=self.obj_num:
#             final_list = selected_list[:self.obj_num]
            return self.object_dict 

        else:
            inter=list(self.select_interesting(list_interests))
#             print(inter)
            dic=dict(zip(inter,[2]*len(inter)))
            self.object_dict={**self.object_dict, **dic}
            
#         print(self.object_dict,len(self.object_dict))   
        
        if len(self.object_dict)>=self.obj_num:
            return self.object_dict
        
        else:
            n=self.obj_num-len(self.object_dict)
            pop=self.select_top_popular(n, list(self.object_dict.keys()))
            dic_pop=dict(zip(pop,[3]*len(pop)))
            self.object_dict={**self.object_dict, **dic_pop}
            
        return self.object_dict
        
    def select_top_popular(self, n, final_list):
        
        _ = conn_obj.open_conn()
        df = conn_obj.get_table('showplace_tbl')
        _ = conn_obj.close_conn()
        df=df.sort_values('popularity', ascending=False)
        df=df[~df['showplace_id'].isin(final_list)]
        return df.iloc[:n]['showplace_id'].values.tolist()
    
    def get_neigbors_by_iso(self, show_id, iso_type):
         
        _ = conn_obj.open_conn()
        df = conn_obj.get_table_cond('neighboring_showplace_tbl', 'showplace_id', show_id)
        _ = conn_obj.close_conn()
        val=df[df['isochrone_type']==iso_type].values.tolist()
        return val
    
    def get_iso_by_showplace(self, show_id1, show_id2):
        
        _ = conn_obj.open_conn()
        df = conn_obj.get_table_cond('neighboring_showplace_tbl', 'showplace_id', show_id1)
        _ = conn_obj.close_conn()
        iso_list=[iso for lis, iso in zip(df['list_showplaces'],df['isochrone_type']) if show_id2 in lis]
        if len(iso_list)>0:
            return iso_list[0]
        else:
            return  None
    
    def select_interesting (self,  list_interests):
        
        _ = conn_obj.open_conn()
        df = conn_obj.get_table('showplace_tbl')
        _ = conn_obj.close_conn()
        
        ind = (
                [ i for i,v in df['k1'].items() if v in list_interests ]
                +
                [ i for i,v in df['k2'].items() if v in list_interests ]
                +
                [ i for i,v in df['k3'].items() if v in list_interests ]
                +
                [ i for i,v in df['k4'].items() if v in list_interests ]
                +
                [ i for i,v in df['k5'].items() if v in
                 list_interests ]
               )
#         ind=[i for i in ind_1 if i in ind_2]
        
        list_obj = df.loc[ind]['showplace_id'].values.tolist()
        
        return list_obj
    
    def get_coords ( self, list_obj):
        _ = conn_obj.open_conn()
        df = conn_obj.get_table('showplace_tbl')
        _ = conn_obj.close_conn()
        list_obj = df[df['showplace_id'].isin(list_obj)]
    
    def get_graph(self):
        
        self.final_list=list(self.object_dict.keys())
        
        list_gr = [(self.final_list[i],self.final_list[j],get_iso_by_showplace(self.final_list[i],self.final_list[j])) for i in range(len(self.final_list)) for j in range(i+1,len(self.final_list))]
        list_gr1 = [ i for i in list_gr]
        self.G = nx.Graph()
        self.G.add_weighted_edges_from(list_gr1)
#         print(self.G.edges())
#         return G
    
    def get_next_place(self, show_id, adj_mat, date, hour, dens_obj):
        
        list_next_var=adj_mat[adj_mat[show_id]>0][show_id].index
#         print('list_next_var',list_next_var)
        if len(list_next_var)> 0:

            sel_list=[i for i in list_next_var if self.object_dict[i] == 1]
            if len(sel_list)>0:
                next_show=sel_list[0]   
            else:
                
                dict_dens={ dens_obj.get_hour_cnt(date, hour, i):i  for i in  list_next_var }
                
                next_show=dict_dens[min(dict_dens.keys())]

            time_to_get=adj_mat.loc[next_show][show_id]

        else:

            col_num=[n+1 for n,i in enumerate(adj_mat.columns) if i==show_id ][0]
            next_show=adj_mat.columns[col_num]
            time_to_get=90

        return next_show, time_to_get
    

ua=user_anketa(route_id, conn_obj)
it=itinerary(ua)
it.get_start(ua)
it.get_graph()
it.get_dict_days(ua, dens_obj)

#возвращаем
it.itinerary